# HW1

### Loading the data from cwd with csv module

In [ ]:
import csv
import numpy as np
import pixiedust
from collections import defaultdict

In [ ]:
train = list(csv.reader(open('income.train.txt.5k', 'r')))
dev = list(csv.reader(open('income.dev.txt', 'r')))
# print('train type: {}'.format(type(train)))
# print('dev type: {}'.format(type(dev)))

### Writing to a csv for easier handling

In [ ]:
with open('income.train.txt.5k', 'r') as readTrainTxt:
    rows = (csv.reader(readTrainTxt))
    with open('income.train.csv', 'w') as writeTrainCSV:
        writer = csv.writer(writeTrainCSV)
        for row in rows:
            writer.writerow(row)

### Handling of data text file without csv

In [ ]:
with open('income.train.txt.5k', 'r') as trainFile:
    trainTxt = trainFile.readlines()
    with open('income.train.csv', 'w') as csvTrainFile:
        #Adding the headers
        csvTrainFile.write('age,sector,education,marital-status,occupation,race,gender,hours-per-week,country-of-origin,target\n')
        for line in trainTxt:
            csvTrainFile.write(line)

## Part 1 - Data Pre-processing (Feature Map) 

In [ ]:
trainRecs = 0
trainPos = 0
for age, sector, ed, ms, occ, race, gender, hours, country, target in train:
#     print("age: {} sector: {} ed: {} ms: {} occ: {} -----------target: {}".format(age, sector, ed, ms, occ, target))
    trainRecs += 1
    trainPos += 1 if target == ' >50K' else 0
print('number of records in training set: {}'.format(trainRecs))
print('number of positive records in training set: {}'.format(trainPos))
print('percentage of positive records in training set: {}'.format(trainPos/trainRecs*100))

### 1) What are the positive % of training data?

Total number of records in training data = 5000

Total number of positive data = 1251

% of positive data = ~25%

In [ ]:
devRecs = 0
devPos = 0
for age, sector, ed, ms, occ, race, gender, hours, country, target in dev:
#     print("age: {} sector: {} ed: {} ms: {} occ: {} -----------target: {}".format(age, sector, ed, ms, occ, target))
    devRecs += 1
    devPos += 1 if target == ' >50K' else 0
print('number of records in dev set: {}'.format(devRecs))
print('number of positive records in dev set: {}'.format(devPos))
print('percentage of positive records in dev set: {}'.format(devPos/devRecs*100))

### 2) Youngest and oldest ages in training data?

### Least and most amount of hours per week do people in this set work? 

In [ ]:
ageTrain = [age for (age, sector, ed, ms, occ, race, gender, hours, country, target) in train]
hoursTrain = [hours for (age, sector, ed, ms, occ, race, gender, hours, country, target) in train]
# print(ageTrain)
print('min age: {}'.format(min(ageTrain)))
print('max age: {}'.format(max(ageTrain)))
# print('---------------------------------')
# print(hoursTrain)
print('min hours: {}'.format(min(hoursTrain)))
print('max hours: {}'.format(max(hoursTrain)))

In [ ]:
def getData(fileName):
    with open(fileName) as trainTxt:
        train = trainTxt.readlines()

        #Getting all data except the target column
        data = list(map(lambda s : s.strip().split(', '), train))
    return data

### Extracting the values of fields in lists

In [ ]:
def getAllData(fileName):
    with open(fileName) as trainTxt:
        train = trainTxt.readlines()

    #Getting all data except the target column
    data = list(map(lambda s : s.strip().split(', '), train))
        
    sector = [line[1] for line in data]
    education = [line[2] for line in data]
    marital = [line[3] for line in data]
    occ = [line[4] for line in data]
    race = [line[5] for line in data]
    gender = [line[6] for line in data]
    country = [line[8] for line in data]
    
    age = [x/50 for x in list(map(int,[line[0] for line in data]))]
    hours = [x/50 for x in list(map(int,[line[7] for line in data]))]
    
    return list(zip(age, sector, education, marital, occ, race, gender, country, hours))

In [ ]:
def getCategoricalFeaturesData(data):
    sector = [line[1] for line in data]
    education = [line[2] for line in data]
    marital = [line[3] for line in data]
    occ = [line[4] for line in data]
    race = [line[5] for line in data]
    gender = [line[6] for line in data]
    country = [line[8] for line in data]
    
    return list(zip(sector, education, marital, occ, race, gender, country))

In [ ]:
def getNumericalFeaturesData(data):
    age = [x/50 for x in list(map(int,[line[0] for line in data]))]
    hours = [x/50 for x in list(map(int,[line[7] for line in data]))]
    
    return list(zip(age, hours))

In [ ]:
def getTarget(data):
    return [line[-1] for line in data]

In [ ]:
trainTarget = getTarget(getData('income.train.txt.5k'))

In [ ]:
trainCategoricalData = getCategoricalFeaturesData(getData('income.train.txt.5k'))

In [ ]:
trainNumericalData = getNumericalFeaturesData(getData('income.train.txt.5k'))

We've now encoded all the features in training data according to index featureMapping. 

Each of these rows can be reverse-looked-up in featureMapping to retrieve their values. 

However, these values are not yet binarized. This is being done in following section. 

### 3) Why do we need to categorize all categorical fields? 

Fields or their values have no meaning for the model unless they're all values of a given field are encoded using a standard scheme for representation purpose. Hence, all categorical fields are encoded to represent the spectrum of values assumed in rows of the dataset. 

In other words, age, sector, education, etc are fields having varied values which needs to be encoded using a standard scheme for model to understand. That is, age=30 or age=56 should be represented as (0,'30') and (0,'56') respectively. Similarly, sector=Private and sector='State-gov' is represented as (1,'Private') and (1,'State-gov') respectively. 

In this way, the model realized that features encountered with first index as 0 represent age, 1 represents sector, 2 represents education, 3 represents marital-status, 4 represents occupation, 5 represents race, 6 represents gender, 7 represents number-of-hours-per-week and 8 represents country.  

### 4) Without considering _age_ and _hours_, what is the maximum possible Euclidean and Manhattan distances between two training examples? 

Barring age, hours and target fields there are seven fields which can be completely different. Based on this, we compute maximum Euclidean and Manhattan distances as follows: 

Euclidean Distance = $\sqrt{(1-0)^2 + (1-0)^2 + (1-0)^2 + (1-0)^2 + (1-0)^2 + (1-0)^2 + (1-0)^2} = \sqrt{7}$

Manhattan Distance = $\mid(1-0)\mid+\mid(1-0)\mid = 7$

### 5) Why do we not want to binarize _age_ and _hours_? What if we did? How should we define the distances on these two dimensions so that each field has equal weight? 

### 6) How many features do you have in total? How many features do you allocate for each of the 9 fields? 

In [ ]:
featureMapping = {}

for row in data:
    for j, x in enumerate(row):
        feature = (j,x)
        if feature not in featureMapping:
            featureMapping[feature] = len(featureMapping)
        elif feature not in tempMap:
            tempMap[feature] = len(tempMap)
print("featureMapping: {}".format(featureMapping))
print('len of featureMapping: {}'.format(len(featureMapping)))

### There are 230 features including age and number-of-hours per week. 

In [ ]:
featureCount = defaultdict(list)
for featureIndex, featureValue in featureMapping.keys():
    featureCount[featureIndex].append(featureValue)
    
print(featureCount)
print([(x, len(featureCount[x])) for x in featureCount.keys()])

| Field |Field Index | Number of features |
|-------|---------|-----------|
|Age | 0 | 67|
|Sector | 1 |  7|
|Education | 2 | 16|
|Marital-Status | 3 | 7|
|Occupation | 4 | 14|
|Race | 5 | 5|
|Gender | 6 | 2|
|Hours-per-week | 7 | 73|
|Country | 8 | 39|

### 7) How many features would you have if you binarize all the fields? 

In [ ]:
def getFeatureMapAndBinarizedData(data):
    featureMapping = {}
    binarizedData = []
    
    for row in data:
        newRow = []
        for j,x in enumerate(row):
            if isinstance(x,str):
                feature = (j,x)
                if feature not in featureMapping:
                    featureMapping[feature] = len(featureMapping)
                newRow.append(featureMapping[feature])
            else:
                newRow.append(x)
        binarizedData.append(newRow)
    return featureMapping, binarizedData

In [ ]:
featureMapping, unbinarizedData = getFeatureMapAndBinarizedData(getAllData('income.train.txt.5k'))

print("featureMapping: {}".format(featureMapping))
# print('Features not in featureMapping: {}'.format(sorted(featureMapping, key=lambda s : s[0])))
# print('len of featureMapping: {}'.format(len(featureMapping)))
print('---------------------')
print('binarizedData: {}'.format(unbinarizedData))

In [ ]:
d = getAllData('income.dev.txt')
print(d[0])
for row in d:
    print(row)
    for j,x in enumerate(row):
        if isinstance(x,str):
            if (j,x) in featureMapping.keys():
                print('{},{} is there in featuremap'.format(j,x)) 
            else:
                print('-----------{},{} is not there in featuremap'.format(j,x)) 

In [ ]:
def getTranslatedOnFeatureMap(featureMapping, data):
    translatedData = []
    
    for row in data:
        newRow = []
        for j,x in enumerate(row):
            feature = (j,x)
            if feature in featureMapping.keys():
                newRow.append(featureMapping[feature])
            else:
                newRow.append(x)
        translatedData.append(newRow)
    return translatedData

### Calculating Euclidean distance

In [ ]:
np.shape(binarizedData)

In [ ]:
finalData = np.zeros((5000,92))

In [ ]:
def getBinarizedData(unbinarizedData, numberOfFeatures):
    finalData = np.zeros((len(unbinarizedData), numberOfFeatures))
    for c, row in enumerate(unbinarizedData):
        for i,x in enumerate(row):
            if isinstance(x,int):
                finalData[c][i] = 1
            elif isinstance(x,float):
                finalData[c][i] = x
    return finalData

In [ ]:
finalData = getBinarizedData(unbinarizedData)

In [ ]:
print(np.linalg.norm(finalData[1] - finalData[2]))

In [ ]:
finalData.shape

In [ ]:
[(i,j) for (i,j) in enumerate(finalData)]

In [ ]:
eucTrain = defaultdict(int)
for i, d in enumerate(finalData):
    eucTrain[i] = np.linalg.norm(d)
print(eucTrain)

#### Gentle introduction to norms.

By default np.linalg.norm gives L2 norm that is Euclidean distance wrt origin that is sqrt(sum(square of difference of distance of point from origin))

In [ ]:
# euc[i] = np.linalg.norm(d)
math.sqrt(sum([x**2 for x in finalData[0]]))
# math.sqrt(sum([x**2 for x in np.arange(9)-4]))

In [ ]:
# print(finalData.shape)
# print(finalData[2].shape)
print(np.linalg.norm(finalData - finalData[2]))
print(finalData - finalData[2])
print((finalData - finalData[2]).shape)

### Try linalg.norm(finalData - finalData[1]) and see if it's equal to manually calculated Euclidean distance between the two. 

In [ ]:
print(np.linalg.norm(finalData - finalData[2]))

## 2) *k*-Nearest Neighbor Classification

### 1. Implementing basic *k*-NN Classifier (with Euclidean distance)

#### q1) Is there any work in training after feature map? - No

#### q2) What's the time complexity *k*-NN to test one example (dimensionality *d*, dize of training set |*D*|)

#### q3) Do you really need to sort the distances first and then choose the top *k*?

### 2. Why *k* in *k*-NN has to be an odd number? 

To get a majority vote in case of equal distribution of votes between neighbors, having odd number of *k* is necessary. 

### 3. Evaluate *k*-NN on the dev set and report the error rate and predicted positive rate for k = 1,3,5,7,9,99,999,9999.

In [ ]:
dev = getAllData('income.dev.txt')
print(len(dev))

In [ ]:
print(dev[0:5])

In [ ]:
devFeatureMap, unbinarizedDevData = getFeatureMapAndBinarizedData(dev)

# print(devFeatureMap)
# print('----------------------------------------------')
print(unbinarizedDevData)

In [ ]:
finalDevData = np.zeros((1000,92))

In [ ]:
finalDevData = getBinarizedData(unbinarizedDevData)
print(finalDevData.shape)

In [ ]:
def getEuclideanDistances(binarizedData, binarizedDevData, k):
    eucs = defaultdict(int)
    for c,devRow in enumerate(binarizedDevData):
        dist = []
        for x,tRow in enumerate(binarizedData):
            dist.append((x,np.linalg.norm(np.array(tRow)-np.array(devRow))))
        eucs[c] = sorted(dist, key = lambda s : s[1])[:k]
    return eucs

In [ ]:
eucs = getEuclideanDistances(binarizedData, binarizedDevData, 1)
print(eucs)

In [ ]:
# trainIncome = getIncome('income.train.txt.5k')
# print(list(getIncome('income.train.txt.5k')))
# print(list(getTarget(getData('income.train.txt.5k'))))

devIncome = getTarget(getData('income.dev.txt'))

In [ ]:
l = sorted(eucs.values(),key = lambda s : s[0][1])
print(l[0])
print(l[-1])

In [ ]:
def getPredictions(eucs):
    predictions = []
#     actuals = []
    for rowIndex,dist in eucs.items():
#         actualLabel = devIncome[rowIndex][1]
        predictedLabels = [trainIncome[i[0]][1] for i in dist]
        predictedLabel = Counter(predictedLabels).most_common(1)[0][0]
        predictions.append(predictedLabel)
#         actuals.append(actualLabel)

#     return predictions, actuals
    return predictions

In [ ]:
predictions = getPredictions(eucs)
actuals = getTarget(getData('income.dev.txt'))

error = 0
for i in range(len(predictions)):
    if actuals[i] != predictions[i]:
        error+=1

# print('error rate for k={} is {}'.format(k,error))
print('k={} error rate: {}% Positive %: {}% is '.format(1,error/10, Counter(predictions)['>50K']/10))

In [ ]:
def getEucDist(trainRow, testRow):
    n = len(trainRow)
    return math.sqrt(sum([ (np.array(trainRow[i]) - np.array(testRow[i])) **2 for i in range(n)] ))

In [ ]:
print(getEucDist([0,0], [1,1]))
print(np.linalg.norm(np.array([[0,0], [1,1]])-np.array([1,1])))
print(np.linalg.norm(np.array([[0,0], [1,1]])-np.array([1,1]), axis=1 ))
print(np.linalg.norm(np.array([[0,0], [1,1], [2,2], [3,3]])-np.array([1,1]), axis = 1))

In [ ]:
# print(unbinarizedDevData[0])
# print(unbinarizedTrainData[4872])
# print(unbinarizedTrainData[4787])
# print(unbinarizedTrainData[2591])
# print(getEucDist(binarizedDevData[0], binarizedTrainData[4872]))

In [ ]:
def getTopKElements(l,k):
    topK = []
    i = 0
    
    while len(topK) < k:
#         print('inside ------------')
        topK.append((l[i],i))
        for j in range(i+1,len(l)):
            if topK[i][0] > l[j]:
#                 temp = topK[i][0]
                topK[i] = (l[j],j)
                l[i], l[j] = l[j], l[i]
                
#             print('j: {} top: {} l: {}'.format(j, topK, l))
        i+=1
    return topK

In [ ]:
l = [45,32,1,13,23,56,77,31]
k = 3
print(getTopKElements(l,k))
print(np.partition([45,32,1,13,23,56,77,31],3))
print(np.argpartition([45,32,1,13,23,56,77,31],3))
# print(l)
print(np.sort([45,32,1,13,23,56,77,31]))
print(np.argsort([45,32,1,13,23,56,77,31]))

In [ ]:
def getAccuracy(predictions):
    actuals = getTarget(getData('income.dev.txt'))
    error = 0
    for i in range(len(predictions)):
        if actuals[i] != predictions[i]:
            error+=1
    return error

In [ ]:
def getPredictions(topKElements):
    trainIncome = getTarget(getData('income.train.txt.5k'))
    predictions = []
    
    for i in range(len(topKElements)):
        predictedLabelsIndex = [trainIncome[j] for dist,j in topKElements[i]]
        predictedLabel = Counter(predictedLabelsIndex).most_common(1)[0][0]
        predictions.append(predictedLabel)
    return predictions

In [ ]:
def getL2NormWithoutIndex(train, test):
    dist = []
    for row in test:
        dist.append(np.linalg.norm(train - row, axis = 1))
    return dist

In [ ]:
binarizedTrainData = getBinarizedData(unbinarizedTrainData, 92)
binarizedDevData = getBinarizedData(unbinarizedDevData, 92)
devDistWithoutIndex = getL2NormWithoutIndex(binarizedTrainData, binarizedDevData)
# [getTopKElements(devDistWithoutIndex[d],3) for d in range(len(devDistWithoutIndex[:20]))]
# [sorted(devDistWithoutIndex[d])[:3] for d in range(len(devDistWithoutIndex[:20]))]

In [ ]:
with open('untitled.txt') as f:
    d = f.readlines()

d = list(map(lambda s : s.split(','), d))
d = list(map(float, [d[i] for i in range(len(d))][0]))
print([d[i] for i in [1578,4396,4092]])
# d = list(map(int, d))
# print(len(d))
# [d for d.split(',') in d]